In [ ]:
from EvoForge.Agent.task_splitter import TaskDecompositionAgent 

In [ ]:
agent = TaskDecompositionAgent(model="gpt-4o", session_id="test_task_decomposition1")
agent.clear_memory()

In [ ]:
agent.rewrite_system_message("You decompose task/multiple tasks into one by one subtasks. The current domain is each subtask should be a project setup task such as setting up a github repo or hugging face model. Each task should only be one repo/hugging face program. Plan in very general high level, you must include the full url of the repo/hugging face model.")

In [ ]:
response = agent.stream_return_graph_state("""
Read through this urls: https://huggingface.co/hexgrad/Kokoro-82M
https://huggingface.co/myshell-ai/MeloTTS-Korean
https://huggingface.co/facebook/mms-tts-yor
https://huggingface.co/microsoft/speecht5_tts
https://huggingface.co/suno/bark
https://huggingface.co/facebook/mms-tts-eng
https://huggingface.co/myshell-ai/MeloTTS-Chinese and create files for each to run each TTS models to produce an audio file. I want the text to be a fun joke. if the model does not seem to work even after trying your best, report the issues.
""")

In [ ]:
for step in response["plan"]["steps"]:
    print("task_name: ", step["task_name"])
    print("task_context: ", step["task_context"])

In [ ]:
tasks = response["plan"]["steps"]


from EvoForge import EvoForge  
evoforge_client = EvoForge()
def run_EvoForge_Agent(session:str,task:str):
    agent = evoforge_client.spawn_setup_agent(session)
    agent.clear_memory()
    result = agent.stream_return_graph_state(task)
    return result

from concurrent.futures import ThreadPoolExecutor
with ThreadPoolExecutor() as executor:
    futures = [
        executor.submit(run_EvoForge_Agent, task["task_name"], task["task_context"])
        for task in tasks
    ]
    results = [future.result() for future in futures]
